In [1]:
import os
import re
import pickle
import datetime
import pandas as pd
import numpy as np

from _html_parser import ParsingDataPrepare, TableConstant

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
%load_ext autoreload
%autoreload 2

#### Choose the date for dirs and files

In [3]:
# Start from more time ('26_05_2022' -'27_03_2021'), ('26_03_2021' - '03_11_2013)
dir_data = dir_data_start = '26_05_2022'
dir_data_end = '27_03_2021'

In [4]:
with open('pickle_files/new_events/'+ dir_data +'/df_'+ dir_data_start +'_to_'+ dir_data_end  +'_games', 'rb') as f:
    df_games = pickle.load(f)

In [5]:
df_games

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue
0,1729359,Ligue 1,18,310,26.05.2022 17:00,Auxerre - Saint-Etienne,1:1,top
1,1729393,Serie B,627,310,26.05.2022 18:30,Monza - Pisa,2:1,thr
2,1729388,Eredivisie,596,Cup,26.05.2022 18:00,Vitesse - AZ Alkmaar,2:1,top
3,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0,top
4,1687458,Eliteserien,747,301,26.05.2022 16:00,Rosenborg - Haugesund,3:3,sec
...,...,...,...,...,...,...,...,...
91916,15197308,Primera Division,593,306,28.03.2021 17:00,Nueva Concepcion - Suchitepequez,1:1,thr
91917,15197309,Primera Division,593,306,28.03.2021 20:59,Deportivo San Pedro - Plataneros,2:1,thr
91918,15092974,National Football League,575,307,28.03.2021 03:00,Navua - Rewa,1:2,thr
91919,15092975,National Football League,575,307,28.03.2021 03:00,Ba - Lautoka,1:2,thr


In [6]:
df_games['f_team'] = df_games.game_title.map(lambda x: str(x).split(' - ')[0])
df_games['s_team'] = df_games.game_title.map(lambda x: str(x).split(' - ')[1])

In [7]:
df_games.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,s_team
0,1729359,Ligue 1,18,310,26.05.2022 17:00,Auxerre - Saint-Etienne,1:1,top,Auxerre,Saint-Etienne
1,1729393,Serie B,627,310,26.05.2022 18:30,Monza - Pisa,2:1,thr,Monza,Pisa
2,1729388,Eredivisie,596,Cup,26.05.2022 18:00,Vitesse - AZ Alkmaar,2:1,top,Vitesse,AZ Alkmaar


#### Add competitions data 

In [8]:
with open('pickle_files/new_events/'+ dir_data +'/df_comp_tabs_'+ dir_data_start +'_all_lgs', 'rb') as f:
    df_comp_tables = pickle.load(f)

In [9]:
df_comp_tables.rename(columns={'team_name':'f_team'}, inplace=True)

In [10]:
# Duble column for get data for each team
df_comp_tables['s_team'] = df_comp_tables.f_team

In [11]:
df_comp_tables.head()

,f_team,team_id,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id,s_team
0,Japan U23,10622,1,3,3,0,0,7,1,+6,9,1016,298,Japan U23
1,Mexico U23,9209,2,3,2,0,1,8,3,+5,6,1016,298,Mexico U23
2,France U23,10739,3,3,1,0,2,5,11,-6,3,1016,298,France U23
3,South Africa U23,10615,4,3,0,0,3,3,8,-5,0,1016,298,South Africa U23
4,South Korea U23,10609,1,3,2,0,1,10,1,+9,6,1016,298,South Korea U23


In [12]:
df_fcom = pd.merge(left=df_games, right=df_comp_tables[['f_team','comp_id','season_id','pos', 'pts']], on=['f_team','comp_id','season_id'], how='left')
df_fcom.rename(columns={'pos':'pos_f', 'pts':'pts_f'}, inplace=True)

In [13]:
df_scom = pd.merge(left=df_games, right=df_comp_tables[['s_team','comp_id','season_id','pos', 'pts']], on=['s_team','comp_id','season_id'], how='left')
df_scom.rename(columns={'pos':'pos_s', 'pts':'pts_s'}, inplace=True)

In [14]:
df_games_tb = pd.merge(df_fcom, df_scom[['game_id', 'pos_s','pts_s']], how='left', left_on='game_id', right_on='game_id')

In [15]:
df_games_tb.pos_f = pd.to_numeric(df_games_tb.pos_f, errors='coerce')
df_games_tb.pos_s = pd.to_numeric(df_games_tb.pos_s, errors='coerce')
df_games_tb.pts_f = pd.to_numeric(df_games_tb.pts_f, errors='coerce')
df_games_tb.pts_s = pd.to_numeric(df_games_tb.pts_s, errors='coerce')

In [16]:
df_games_tb['posf_poss'] = df_games_tb[['pos_f', 'pos_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 
df_games_tb['ptsf_ptss'] = df_games_tb[['pts_f', 'pts_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 

In [17]:
df_games_tb.drop(columns=['s_team'], inplace=True)

In [18]:
df_games_tb.drop_duplicates(subset=['game_id'], inplace=True)

In [19]:
df_games_tb.reset_index(drop=True, inplace=True)

In [20]:
df_games_tb.posf_poss.value_counts(dropna=False).head(4)

None     16922
1 - 3      706
1 - 2      692
1 - 4      683
Name: posf_poss, dtype: int64

In [21]:
df_games_tb.posf_poss.value_counts(dropna=False).tail(4)

3 - 36     1
17 - 37    1
27 - 35    1
21 - 27    1
Name: posf_poss, dtype: int64

In [22]:
df_comp_tables.pos = pd.to_numeric(df_comp_tables.pos, errors='coerce')
df_comp_tables.pts = pd.to_numeric(df_comp_tables.pts, errors='coerce')

df_max_pos = df_comp_tables.groupby(['comp_id', 'season_id'])['pos'].max().to_frame().reset_index()  
df_max_pts = df_comp_tables.groupby(['comp_id', 'season_id'])['pts'].max().to_frame().reset_index() 

df_max_pos.rename(columns={'pos':'max_pos'}, inplace=True)
df_max_pts.rename(columns={'pts':'max_pts'}, inplace=True)

In [23]:
df_max_pos.head(3)

,comp_id,season_id,max_pos
0,1016,298,4
1,1019,310,4
2,1035,306,11


In [24]:
df_max_pts.head(3)

,comp_id,season_id,max_pts
0,1016,298,9
1,1019,310,16
2,1035,306,28


In [25]:
df_games_tb = df_games_tb.merge(df_max_pos, how='left', left_on=['comp_id', 'season_id'],  right_on=['comp_id', 'season_id'])
df_games_tb = df_games_tb.merge(df_max_pts, how='left', left_on=['comp_id', 'season_id'],  right_on=['comp_id', 'season_id'])

In [26]:
df_games_tb['fs_pos_max'] = df_games_tb[['posf_poss', 'max_pos']].apply(lambda x: str(x[0])+ ', ' + str(x[1]).replace('.0', '') \
                                                                                                                  if x[0] != None else None, axis=1)
df_games_tb['fs_pts_max'] = df_games_tb[['ptsf_ptss', 'max_pts']].apply(lambda x: str(x[0])+ ', ' + str(x[1]).replace('.0', '') \
                                                                                                                  if x[0] != None else None, axis=1)

In [27]:
df_games_tb.drop(columns=['posf_poss', 'ptsf_ptss', 'max_pts'], inplace=True)

#### Add games events - don't use

In [28]:
# with open('pickle_files/new_events/'+ dir_data +'/df_evns_'+ dir_data_start +'_to_'+ dir_data_end, 'rb') as f:
#     df_events = pickle.load(f)

In [29]:
# df_events.event_desc = df_events.event_desc.map(lambda x: [y[0]+y[-1] for y in x])

In [30]:
# print('shape - ', df_events.shape[0])
# df_events.head(2)

In [31]:
# # Check max len stats_dict col for new names df's columns
# maxlen_stats_dict = df_events.stats_dict.map(lambda x: len(x))
# maxlen_stats_dict.max()

In [32]:
# maxlen_stats_dict[maxlen_stats_dict == 19].head(2)

In [33]:
# '2592' index with maxlen 'stats_dict' column
# col_names = [x[0] for x in df_events.stats_dict[2592]]

In [34]:
# str(col_names)

In [35]:
# for col in col_names:
#     df_events[col] = df_events.stats_dict.map(lambda x: ''.join([str(y[1]) + '-' + str(y[2]) for y in x if y[0] == col]))

In [36]:
# df_events.drop(columns=['stats_dict', 'xG','Shots on target', 'Shots Blocked', 'Offsides', 'Pass', 'Pass percentage %', 'Free Kicks', 
#                         'Crosses', 'Tackles','Throw Ins'], inplace=True)

In [37]:
# col_names = ['tm_1', 'tm_x', 'tm_2', 'ls_25', 'mr_25']

# for ind, col in enumerate(col_names):
#     df_events[col] = df_events.bets.map(lambda x: x[1][ind] if x[1] != [] else None)

In [38]:
# df_events.tm_1 = pd.to_numeric(df_events.tm_1, errors='coerce')
# df_events.tm_2 = pd.to_numeric(df_events.tm_2, errors='coerce')

In [39]:
# df_events.drop(columns=['bets'], inplace=True)

In [40]:
# df_events.columns = df_events.columns.str.lower()

In [41]:
# df_events.rename(columns={'possession %':'poss', 'yellow cards':'yl_crd', 'red cards':'rd_crd', 'attacks (dangerous)':'att_dang'}, inplace=True)

In [42]:
# print('shape - ', df_events.shape[0])
# df_events.head(3)

In [43]:
# main_ligues = pd.read_csv('csv_files/main_ligues.csv')
# main_ligues = main_ligues.top_ligues.squeeze()
# df_main = df[df.ligue_header.isin(main_ligues)].reset_index(drop=True)
# df_main.shape
# (353909, 8)

#### Concatinate dataframes with best and tables position coefficients

In [44]:
df_power_coefs = df_games_tb[df_games_tb.max_pos >= 8].copy().reset_index(drop=True)
df_power_coefs.shape[0]

74587

In [45]:
# df_gms_events = df_games_tb.merge(df_events, left_on='game_id', right_on='game_id')

In [46]:
# df_gms_events.shape[0]

In [47]:
# df_games_tb_not_events = df_games_tb[~df_games_tb.game_id.isin(df_gms_events.game_id)]

In [48]:
# df_games_tb_not_events.shape[0]

In [49]:
# df_games_events = pd.concat([df_games_tb_not_events, df_gms_events], axis=0).reset_index(drop=True)

In [50]:
# df_games_events.replace(to_replace=[np.nan], value=None, inplace=True)
# df_games_events = df_games_events.fillna('',).replace([''], [None]).copy()

In [51]:
# df_games_events.tm_2.value_counts(dropna=False).head(3)

In [52]:
# df_games_events.fs_pos_max.value_counts(dropna=False).head(3)

In [53]:
# df_power_coefs = df_games_events[(~df_games_events.tm_1.isna()) | (df_games_events.max_pos >= 8)].copy()

In [54]:
# df_power_coefs.tm_1.value_counts(dropna=False)

In [55]:
df_power_coefs.max_pos.value_counts(dropna=False)

20.0    16303
16.0     9883
18.0     8383
12.0     7766
10.0     5208
21.0     4811
14.0     4245
17.0     3043
22.0     2990
24.0     1979
8.0      1728
19.0     1501
23.0     1442
9.0      1256
13.0     1216
11.0     1073
15.0     1004
26.0      325
32.0      264
37.0      167
Name: max_pos, dtype: int64

In [56]:
df_power_coefs.drop(columns=['max_pos'], inplace=True)

In [57]:
print('shape - ', df_power_coefs.shape[0])
df_power_coefs.tail()

shape -  74587


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max
74582,15197084,Birinci Divizion,476,306,28.03.2021 11:00,MOIK - Kapaz,4:1,thr,MOIK,10.0,20.0,8.0,28.0,"10 - 8, 10","20 - 28, 64"
74583,15197083,Birinci Divizion,476,306,28.03.2021 11:00,Neftchi II - Sumqayıt II,6:2,thr,Neftchi II,1.0,64.0,9.0,22.0,"1 - 9, 10","64 - 22, 64"
74584,15092974,National Football League,575,307,28.03.2021 03:00,Navua - Rewa,1:2,thr,Navua,7.0,9.0,2.0,25.0,"7 - 2, 8","9 - 25, 30"
74585,15092975,National Football League,575,307,28.03.2021 03:00,Ba - Lautoka,1:2,thr,Ba,3.0,22.0,1.0,30.0,"3 - 1, 8","22 - 30, 30"
74586,15092976,National Football League,575,307,28.03.2021 03:00,Nadi - Suva,1:0,thr,Nadi,6.0,15.0,4.0,21.0,"6 - 4, 8","15 - 21, 30"


#### Power coefficient and statistic

In [58]:
# pow_coef = 1.9
# df_power_coefs['f_pow_bet'] = df_power_coefs[['tm_1', 'tm_2']].apply(lambda x: ParsingDataPrepare.roles_determinate([x[0], x[1]], pow_coef, 'min'), axis=1)

In [59]:
# df_power_coefs.f_pow_bet.value_counts(dropna=False)

In [60]:
# pow_coef = 1.95
# df_power_coefs['f_pow_pos'] = df_power_coefs[['pos_f', 'pos_s']].apply(lambda x: ParsingDataPrepare.roles_determinate([x[0], x[1]], pow_coef, 'min'), axis=1)

In [61]:
# df_power_coefs.f_pow_pos.value_counts(dropna=False)

In [62]:
pow_coef = 2
df_power_coefs['f_pow_pts'] = df_power_coefs[['pts_f', 'pts_s']].apply(lambda x: ParsingDataPrepare.roles_determinate([x[0], x[1]], pow_coef, 'max'), axis=1)

In [63]:
df_power_coefs.f_pow_pts.value_counts(dropna=False)

Neu    62256
Pre     6172
Fav     6159
Name: f_pow_pts, dtype: int64

In [64]:
df_power_coefs['f_pow'] = df_power_coefs.f_pow_pts

In [65]:
# df_power_coefs.f_pow = df_power_coefs[['f_pow', 'f_pow_bet']].apply(lambda x: x[1] if x[0] == None and x[1] == 'Fav' else x[0], axis=1)

In [66]:
# df_power_coefs.f_pow = df_power_coefs[['f_pow', 'f_pow_bet']].apply(lambda x: x[1] if x[0] == None and x[1] == 'Pre' else x[0], axis=1)

In [67]:
df_power_coefs.f_pow.value_counts(dropna=False)

Neu    62256
Pre     6172
Fav     6159
Name: f_pow, dtype: int64

In [68]:
df_power_coefs.drop(columns='f_pow_pts', inplace=True)

In [69]:
df_fav_pre = df_power_coefs[df_power_coefs.f_pow.isin(['Fav', 'Pre'])].copy()

In [70]:
df_fav_pre['desc'] = 'fav_pre'

In [71]:
df_fav_pre.reset_index(drop=True, inplace=True)

In [72]:
print('shape - ', df_fav_pre.shape[0])
df_fav_pre.tail(3)

shape -  12331


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc
12328,15109590,Thai League 1,718,298,28.03.2021 11:00,Rayong - Chonburi,1:1,thr,Rayong,16.0,15.0,12.0,32.0,"16 - 12, 16","15 - 32, 77",Pre,fav_pre
12329,15197083,Birinci Divizion,476,306,28.03.2021 11:00,Neftchi II - Sumqayıt II,6:2,thr,Neftchi II,1.0,64.0,9.0,22.0,"1 - 9, 10","64 - 22, 64",Fav,fav_pre
12330,15092974,National Football League,575,307,28.03.2021 03:00,Navua - Rewa,1:2,thr,Navua,7.0,9.0,2.0,25.0,"7 - 2, 8","9 - 25, 30",Pre,fav_pre


In [73]:
# df_statistic = df_power_coefs[~df_power_coefs.event_mins.isna()].copy()

In [74]:
# df_statistic = df_statistic[df_statistic.event_mins.map(lambda x: len(x) > 0)].copy()

In [75]:
# df_statistic['desc'] = 'stat'

In [76]:
# df_statistic.reset_index(drop=True, inplace=True)

In [77]:
# print('shape - ', df_statistic.shape[0])
# df_statistic.head(2)

In [78]:
# df_for_cities = pd.concat([df_fav_pre, df_statistic]).reset_index(drop=True)
df_for_cities = df_fav_pre.copy()

In [79]:
df_for_cities.shape

(12331, 17)

In [80]:
# stop

#### Add cities and countries

In [81]:
with open('pickle_files/cities_countries/df_countries_champ_ids', 'rb') as f:
    df_countries = pickle.load(f)

In [82]:
df_countries.drop_duplicates(subset=['comp_id', 'country'], inplace=True)
df_countries.reset_index(drop=True, inplace=True)

In [83]:
df_countries.head(2)

,comp_id,country
0,12,England
1,13,Russia


In [84]:
df_countries.shape

(337, 2)

In [85]:
str(set(df_for_cities[~df_for_cities.comp_id.isin(df_countries.comp_id)].comp_id))

'set()'

In [86]:
df_for_cities = df_for_cities.merge(df_countries, how='left', left_on='comp_id', right_on='comp_id')

In [87]:
df_for_cities.shape

(12331, 18)

In [88]:
df_for_cities.country.value_counts(dropna=False).head()

Spain      1155
England     823
Germany     670
Italy       670
Russia      573
Name: country, dtype: int64

In [89]:
# df_for_cities['city'] = df_for_cities.city_country.map(lambda x: str(x).split(', ')[0] if len(str(x).split(', ')) == 2 else None)

In [90]:
# df_for_cities.city.value_counts(dropna=False)

In [91]:
# df_for_cities.drop(columns=['city_country'], inplace=True)

In [93]:
# stop

#### Add new data

In [94]:
with open('pickle_files/teams_data/df_tmd_main_created_29_06_2022', 'rb') as f:
    df_teams_data = pickle.load(f)

In [95]:
print('shape - ', df_teams_data.shape[0])
df_teams_data.head(3)

shape -  4151


,team,team_id,country,city
0,AaB,None,Denmark,Aalborg
1,Aalesund,None,Norway,Alesund
2,Aarau,None,Switzerland,Aarau


In [96]:
df_teams_data.rename(columns={'team':'f_team'}, inplace=True)

In [97]:
print('shape - ', df_for_cities.shape[0])
df_for_cities = df_for_cities.merge(df_teams_data, how='left', left_on=['f_team', 'country'], right_on=['f_team', 'country'])
print('shape - ', df_for_cities.shape[0])

shape -  12331
shape -  12331


In [98]:
df_for_cities.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city
0,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0,top,FC Minsk,4.0,19.0,15.0,4.0,"4 - 15, 16","19 - 4, 23",Fav,fav_pre,Belarus,540,Minsk
1,1687696,OBOS liga,668,301,26.05.2022 15:00,Skeid - Sandnes Ulf,0:1,thr,Skeid,16.0,3.0,4.0,18.0,"16 - 4, 16","3 - 18, 23",Pre,fav_pre,Norway,10294,Oslo
2,1687694,OBOS liga,668,301,26.05.2022 16:00,Kongsvinger - Brann,1:3,thr,Kongsvinger,13.0,7.0,1.0,23.0,"13 - 1, 16","7 - 23, 23",Pre,fav_pre,Norway,None,Kongsvinger


In [99]:
df_for_cities.city.value_counts(dropna=False)[:3]

NaN             3121
Buenos Aires      59
Yerevan           58
Name: city, dtype: int64

In [111]:
df_for_geo_coords_26_05_2022 = df_for_cities[~df_for_cities.city.isna()].copy().reset_index(drop=True)
print('shape - ', df_for_geo_coords_26_05_2022.shape[0])
df_for_geo_coords_26_05_2022.tail(3)

shape -  9210


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city
9207,15094094,C-League,535,307,28.03.2021 08:30,Asia Europe University - Police Commissary,3:2,thr,Asia Europe University,9.0,13.0,13.0,4.0,"9 - 13, 13","13 - 4, 28",Fav,fav_pre,Cambodia,13409,Kandal
9208,15091817,Primera B Metropolitana,457,307,28.03.2021 19:30,Argentino Quilmes - Defensores Unidos,0:1,sec,Argentino Quilmes,3.0,0.0,7.0,46.0,"3 - 7, 16","0 - 46, 61",Pre,fav_pre,Argentina,12788,Quilmes
9209,15197083,Birinci Divizion,476,306,28.03.2021 11:00,Neftchi II - Sumqayıt II,6:2,thr,Neftchi II,1.0,64.0,9.0,22.0,"1 - 9, 10","64 - 22, 64",Fav,fav_pre,Azerbaijan,NaN,Baku


In [113]:
# file = open('pickle_files/df_for_geo_coords_26_05_2022', 'wb')
# pickle.dump(df_for_geo_coords_26_05_2022, file)  
# file.close()

#### For find teams with na cities

In [100]:
# df_na_cities = pd.DataFrame(df_for_cities[['f_team', 'country']][df_for_cities.city.isna()])

In [101]:
# df_na_cities = df_na_cities.drop_duplicates(subset=['f_team', 'country'], keep='first').reset_index(drop=True)

In [102]:
# df_teams_counts_na = df_for_cities[df_for_cities.city.isna()].f_team.value_counts()

In [103]:
# numb = 3

In [104]:
# len(df_teams_counts_na[df_teams_counts_na >= numb])

In [105]:
# df_teams_counts_na[df_teams_counts_na >= numb].values.sum()

In [106]:
# df_teams_counts_na

In [107]:
# df_teams_counts_na_numb = df_teams_counts_na[df_teams_counts_na >= numb].index

In [108]:
# df_teams_counts_na_numb

In [109]:
# df_teams_cities_na_26_05_2022_third = df_na_cities[df_na_cities.f_team.isin(df_teams_counts_na_numb)].reset_index(drop=True)

In [110]:
# df_teams_cities_na_26_05_2022_third

In [ ]:
# file = open('pickle_files/df_teams_cities_na_26_05_2022_third', 'wb')
# pickle.dump(df_teams_cities_na_26_05_2022_third, file)  
# file.close()